In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pretrained-fasttext/pretrained_word2vec/pretrained_word2vec.txt
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.wv.vectors_vocab.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.trainables.vectors_ngrams_lockf.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.wv.vectors.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.trainables.syn1neg.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.trainables.vectors_vocab_lockf.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text.wv.vectors_ngrams.npy
/kaggle/input/pretrained-fasttext/pretrained_fasttext/pretrained_fasttext/pretrained.fast_text
/kaggle/input/nepsa/NepSA.csv


In [2]:
from pathlib import Path

df= pd.read_csv("/kaggle/input/nepsa/NepSA.csv")
df_copy= df.copy()
df.info()

scratch_output = "./scratch/"
finetuned_output = "./output"
Path(scratch_output).mkdir()
Path(finetuned_output).mkdir()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4707 entries, 0 to 4706
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    4707 non-null   object
 1   Target  4707 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 73.7+ KB


In [3]:
# Tokenizing using nltk and creating a corpus for models to train on

import nltk
from nltk.tokenize import word_tokenize

sentences= df_copy['Text'].apply(lambda x: word_tokenize(x)).tolist()

# Word2Vec scratch

In [4]:
from gensim.models import Word2Vec

scratch = Word2Vec(sentences, vector_size= 300,sg=1 ,negative=10 ,window= 15, min_count=3,alpha=0.0025, epochs=200, workers=4, seed= 42, compute_loss=True)  #sg=1 means using Skip-gram
print(scratch.get_latest_training_loss())

24783234.0


In [5]:
print(f"दलाल: {scratch.wv.most_similar('दलाल')}")
print(f"मुजि: {scratch.wv.most_similar('मुजि')}")
print(f"रन्डि: {scratch.wv.most_similar('रन्डि')}")
print(f"चिक्ने: {scratch.wv.most_similar('चिक्ने')}")
print(f"ठोक्ने: {scratch.wv.most_similar('ठोक्ने')}")
print(f"बलात्कारी: {scratch.wv.most_similar('बलात्कारी')}")
print(f"माचिक्ने: {scratch.wv.most_similar('माचिक्ने')}")
print(f"Similarity between ठोक and हान्नु: {scratch.wv.similarity('ठोक', 'हान्नु')}")

दलाल: [('गोरू', 0.6038889288902283), ('घुसिया', 0.591804563999176), ('पापी', 0.5852213501930237), ('संसद', 0.5716174244880676), ('भारतिय', 0.5635927319526672), ('भुमाफिया', 0.5588367581367493), ('नपाएर', 0.550086498260498), ('रहने', 0.5464518666267395), ('सम्बन्ध', 0.544558584690094), ('डाका', 0.5416148900985718)]
मुजि: [('नगेन्द्र', 0.7136744856834412), ('हरामि', 0.6718366146087646), ('आय', 0.6610497832298279), ('रण्डि', 0.6600621342658997), ('बादी', 0.6586503982543945), ('चम्चा', 0.6574056148529053), ('खालि', 0.6565040946006775), ('प्रचन्डे', 0.6562313437461853), ('बोल्नै', 0.654056191444397), ('लामो', 0.6482406258583069)]
रन्डि: [('रन्दि', 0.8025878667831421), ('राडी', 0.7951111197471619), ('मास्टर', 0.7848541140556335), ('चिक्ने', 0.7755618691444397), ('तेइ', 0.7742782831192017), ('पढ्ने', 0.7729200720787048), ('बान', 0.7703817486763), ('बन', 0.7690048813819885), ('जादा', 0.7669946551322937), ('चिरि', 0.7658504247665405)]
चिक्ने: [('मया', 0.9025090932846069), ('मास्टर', 0.853207349

Saving Word2Vec will be done in txt as Word2Vec doesn't make sub word vectors unlike fasttext. So vectors saved as txt file will suffice.Furthermore the pre-trained Word2Vec model itself is a txt file and not a full model

In [6]:
scratch.wv.save_word2vec_format(os.path.join(scratch_output, 'scratch.txt'), binary=False)

# Pre-trained Word2Vec

In [7]:
from gensim.models import KeyedVectors

pretrained_input = "/kaggle/input/pretrained-fasttext/pretrained_word2vec/pretrained_word2vec.txt"

pretrained = KeyedVectors.load_word2vec_format(pretrained_input, binary=False)

# Fine-Tuned Word2Vec
The pretrained model is a txt file and finetuning without a full model doesn't sound convincing. However a full model from [NPVec1](https://github.com/nowalab/nepali-word-embeddings), specifically the tokenized and stemmed one which fits required case the most doesn't load at all. That's why the alternative being the 300 dimensional Word2Vec [model](https://ieee-dataport.org/open-access/300-dimensional-word-embeddings-nepali-language) which is only a text file of the word vectors. The finetuning procedure will be as stated in this [site](https://czarrar.github.io/Gensim-Word2Vec/) and another [site](https://datascience.stackexchange.com/questions/97568/fine-tuning-pre-trained-word2vec-model-with-gensim-4-0).

In [8]:
finetuned = Word2Vec(vector_size = 300,sg=1 ,negative=10 ,window= 15, min_count=1,alpha=0.0025, epochs=200, workers=4, seed= 42)
finetuned.build_vocab(sentences)

# Adding pre-trained model vocabulary
pretrained_vocab = [[word] for word in pretrained.key_to_index.keys()]
finetuned.build_vocab(pretrained_vocab, update=True)

# Ensuring all the words in the vocabulary are updated during finetuning with vectors of ones
vocab_size = len(finetuned.wv)
finetuned.wv.vectors_lockf = np.ones(vocab_size, dtype=np.float32)

# Load the pre-trained models embeddings
# note: if a word doesn't exist in the pre-trained vocabulary then it is left as is in the original model
finetuned.wv.intersect_word2vec_format(pretrained_input, binary=False, lockf=1.0)
print(len(finetuned.wv))
total_examples = finetuned.corpus_count

# Finally Finetuning
finetuned.train(sentences, total_examples=total_examples, epochs= finetuned.epochs)

519271


(8660367, 9137400)

In [9]:
print(f"दलाल: {finetuned.wv.most_similar('दलाल')}")
print(f"मुजि: {finetuned.wv.most_similar('मुजि')}")
print(f"रन्डि: {finetuned.wv.most_similar('रन्डि')}")
print(f"चिक्ने: {finetuned.wv.most_similar('चिक्ने')}")
print(f"ठोक्ने: {finetuned.wv.most_similar('ठोक्ने')}")
print(f"बलात्कारी: {finetuned.wv.most_similar('बलात्कारी')}")
print(f"माचिक्ने: {finetuned.wv.most_similar('माचिक्ने')}")
print(f"Similarity between ठोक and हान्नु: {finetuned.wv.similarity('ठोक', 'हान्नु')}")

दलाल: [('दलाली', 0.652705729007721), ('दलालहरू', 0.6195130944252014), ('ठग', 0.5988847613334656), ('भुमाफिया', 0.5781099796295166), ('बिचौलिया', 0.5708465576171875), ('पुँजीपति', 0.5636776089668274), ('फटाह', 0.5504918694496155), ('गेरो', 0.5478551387786865), ('कारेकरम', 0.5454437136650085), ('सूरू', 0.5442875623703003)]
मुजि: [('phd', 0.8831667304039001), ('बोग्दा', 0.8724841475486755), ('तेरि', 0.8723926544189453), ('प्रचन्डे', 0.870991587638855), ('मपनी', 0.8708116412162781), ('पुति', 0.8703693151473999), ('हाल्दैन', 0.8702211380004883), ('खोक्क्रो', 0.8700627088546753), ('नहासे', 0.8697423338890076), ('चुतिया', 0.8692623376846313)]
रन्डि: [('चिक्ने', 0.9668002128601074), ('अत्रो', 0.9662339091300964), ('फाल्दिनुनी', 0.9618247747421265), ('तेरिमा', 0.9599411487579346), ('जिउदै', 0.9556483030319214), ('त्रिसुलि', 0.9495136141777039), ('१जन', 0.9491158723831177), ('खाटे', 0.9488843679428101), ('पर्दोइन', 0.9485908150672913), ('हाल्योनि', 0.9483177661895752)]
चिक्ने: [('रन्डि', 0.96680

In [10]:
finetuned.wv.save_word2vec_format(os.path.join(finetuned_output, 'finetuned.txt'), binary=False)

In [11]:
# keeping this at last as this will raise Exception and stop the script. So that all the models are saved before the exception
print(f"दलाल: {pretrained.most_similar('दलाल')}")
print(f"मुजि: {pretrained.most_similar('मुजि')}")
print(f"रन्डि: {pretrained.most_similar('रन्डि')}")
print(f"चिक्ने: {pretrained.most_similar('चिक्ने')}")
print(f"ठोक्ने: {pretrained.most_similar('ठोक्ने')}")
print(f"बलात्कारी: {pretrained.most_similar('बलात्कारी')}")
print(f"माचिक्ने: {pretrained.most_similar('माचिक्ने')}")
print(f"Similarity between ठोक and हान्नु: {pretrained.similarity('ठोक', 'हान्नु')}")

दलाल: [('दलालहरू', 0.6451959013938904), ('दलाली', 0.5998064875602722), ('बिचौलिया', 0.5902517437934875), ('पुँजीपति', 0.5644993782043457), ('ठग', 0.5342469215393066), ('एजेन्ट', 0.5191587805747986), ('पूँजीपति', 0.5182876586914062), ('नोकरशाही', 0.5007683634757996), ('कारोबारी', 0.4855310916900635), ('गिरोह', 0.4851357042789459)]


KeyError: "Key 'मुजि' not present in vocabulary"

As observed the model doesn't have word vectors for profane word, which is expected as the model was trained on new sources and wikipedia.